# The Hubbard model with a UCC ansatz
In this notebook, we will demonstrate how to approximate the ground state of a Hubbard Hamiltonian in tequila using a UCC ansatz. The [Hubbard model](https://en.wikipedia.org/wiki/Hubbard_model) provides a model Hamiltonian in which the system consists of a number of orthogonal interacting sites. In second quantization the Hamiltonian is given by
$$
\begin{equation*}
\mathcal{H} = \sum_{\braket{ij}} t_{ij} \hat{a}^\dagger_i \hat{a}_j + \sum_{i}U_i\hat{n}_{i\uparrow}\hat{n}_{i\downarrow}
\end{equation*}
$$
where $t_{ij}$ is the hopping term between sites $i$ and $j$ and the $U_i$ is the repulsion between electrons on the same site. The $\hat{n}_i$ is the number operator for site $i$ and is given by $\hat{n}_i = \hat{a}_i^\dagger\hat{a}_i$.

In [1]:
import tequila as tq
import openfermion as of
import cirq
import numpy as np

We can use OpenFermion to generate a Hubbard hamiltonian and turn it into a tequila QubitHamiltonian object.

In [2]:
# create the OpenFermion Hubbard hamiltonian in second quantization
openfermion_hamiltonian = of.fermi_hubbard(1, 2, tunneling=1, coulomb=0.5)

# Map the hamiltonian to the qubit space to form a QubitOperator
openfermion_hamiltonian_jw = of.jordan_wigner(openfermion_hamiltonian)

# turn the QubitOperator in a tequila QubitHamiltonian
hamiltonian = tq.QubitHamiltonian.from_openfermion(openfermion_hamiltonian_jw)

Now we need to make the UCC ansatz, in tequila this depends on the Molecule object. However, for the Hubbard model we cannot rely on this, so we need to manually create all excitations. In UCC the order of the excitation operators matters, since the individual excitation operators do not commute with each other. We can once again use the OpenFermion functions to generate this circuit. For the two site Hubbard model in this tutorial, we will only consider singlet excitations.

In [3]:
# first, we start with an initial guess for the parameters of the wave function
# In the singlet case, there are two independent parameters
num_params = of.uccsd_singlet_paramsize(4, 2)

# for the initial guess we supply variables
init = [np.pi/3 for i in range(num_params)]

unitary = of.uccsd_singlet_generator(init, 4, 2)

This ansatz creates only single and double excitation operators in second quantization. However, the order of these operators matters, since information only folws in one direction trough the quantum circuit. According to [Grimsley et al.](https://pubs.acs.org/doi/10.1021/acs.jctc.9b01083) the double excitations shoud come before the single excitations. So here we will separate the single and double excitations into separate circuits and then apply the doubles first. But first we need to transform the unitary using the Jordan Wigner transformation and encode the resulting operator in a quantum circuit. We use the approach outlined by [Anand et al.](https://pubs.rsc.org/en/content/articlelanding/2022/cs/d1cs00932j)

In [4]:
# these functions aid in the creation of the circuit resulting from an exponential of pauli strings.
def build_cnot_ladder(circuit: tq.circuit.circuit.QCircuit, involved_qubits:list) -> tq.circuit.circuit.QCircuit:
    """
    adds a cnot ladder to the given circuit, starts with the first qubit
    
    :param circuit: the quantum circuit of interest
    :param involved_qubits: the qubits that are involved in the rotation
    """
    # the qubits are sorted in ascending order (q0, q1, q2, ...)
    qubits = involved_qubits

    for index, qubit in enumerate(qubits[:-1]):
        circuit += tq.gates.X(control=qubit, target=qubits[index + 1])
    
    return circuit

def build_reverse_cnot_ladder(circuit: tq.circuit.circuit.QCircuit, involved_qubits:list) -> tq.circuit.circuit.QCircuit:
    """
    adds a cnot ladder to the given circuit, starts with the last qubit
    
    :param circuit: the quantum circuit of interest
    :param involved_qubits: the qubits that are involved in the rotation
    """
    # the qubits are sorted in ascending order (q0, q1, q2, ...)
    qubits = involved_qubits
    qubits.sort(reverse=True) # we need them in descending order

    for index, qubit in enumerate(qubits[:-1]):
        circuit += tq.gates.X(control=qubits[index + 1], target=qubit)
    
    return circuit

def create_exponential_map(paulistring: of.ops.operators.qubit_operator.QubitOperator, n_qubits:int) -> tq.circuit.circuit.QCircuit:
    """
    Creates a quantum circuit corresponding to the passed pauli string
    
    :param paulistring: the paulistring you want to turn into an operator
    :param n_qubits: the amount of qubits
    """
    qubits = [i for i in range(n_qubits)]
    circuit = tq.circuit.QCircuit()

    # not all qubits have to be involved in every rotation
    involved_qubits = []

    # the terms are contained in a dictionary
    for element in paulistring.terms.keys():
        for operator in element:
            if operator[1] == "X":
                circuit += tq.gates.H(target=qubits[operator[0]])
            elif operator[1] == "Y":
                circuit += tq.gates.Rx(angle=np.pi/2, target=qubits[operator[0]])
            involved_qubits.append(qubits[operator[0]])
    involved_qubits.sort()
    circuit = build_cnot_ladder(circuit, involved_qubits)
    
    for element in paulistring.terms.keys():
        rotation_angle = -2*paulistring.terms[element].imag
        circuit += tq.gates.Rz(angle=2*rotation_angle, target=involved_qubits[-1])
    
    circuit = build_reverse_cnot_ladder(circuit, involved_qubits)

    # the terms are contained in a dictionary
    for element in paulistring.terms.keys():
        for operator in element:
            if operator[1] == "X":
                circuit += tq.gates.H(qubits[operator[0]])
            elif operator[1] == "Y":
                circuit += tq.gates.Rx(angle=-np.pi/2, target=qubits[operator[0]])
    
    return circuit

In [5]:
unitary_jw = of.jordan_wigner(unitary)
singles, doubles = [], []
for element in unitary_jw.get_operators():
    gates = list(element.terms.keys())[0]
    # single excitation gates always have three single qubit gates
    # doubles always have four, this only holds in this specific example
    if len(gates) == 3:
        singles.append(element)
    else:
        doubles.append(element)

total = doubles + singles

num_qubits = 4
num_electrons = 2
U = tq.circuit.QCircuit()

# initialize the electrons
for electron in range(num_electrons):
    U += tq.gates.X(electron)

# build the excitation gates
for excitation_gate in total:
    partial = create_exponential_map(excitation_gate, num_qubits)
    U += partial

tq.draw(U, backend="cirq")

0: ───X───────X^0.5───@──────────────────────────────@────────X^-0.5───X^0.5───@──────────────────────────────────@────────X^-0.5───H───@─────────────────────────────@────────H───────H───────@──────────────────────────────────@────────H────────X^0.5───@──────────────────────────────@───────X^-0.5───X^0.5───@───────────────────────────────────@───X^-0.5───H───────@──────────────────────────────@───────H────────H───@──────────────────────────────────@───H───X^0.5───@──────────────────────@───X^-0.5───H───@─────────────────────@────────H──────────────────────────────────────────────────────────────────────────────────
                      │                              │                         │                                  │                     │                             │                        │                                  │                         │                              │                        │                                   │                    │           

''

Now that we have the ansatz and the hamiltonian ready, we can calculate the expectation value using the standard tequila functions. However, since the OpenFermion UCCSD generator requires numeric inputs we cannot simply supply tequila variables. As such we will have to write a simple optimize function based on scipy optimization routines.

In [6]:
import scipy.optimize as op

def optimize(input):
    unitary = of.uccsd_singlet_generator(input, 4, 2)
    unitary_jw = of.jordan_wigner(unitary)
    singles, doubles = [], []
    for element in unitary_jw.get_operators():
        gates = list(element.terms.keys())[0]
        # single excitation gates always have three single qubit gates
        # doubles always have four, this only holds in this specific example
        if len(gates) == 3:
            singles.append(element)
        else:
            doubles.append(element)

    total = doubles + singles

    num_qubits = 4
    num_electrons = 2
    U = tq.circuit.QCircuit()

    # initialize the electrons
    for electron in range(num_electrons):
        U += tq.gates.X(electron)

    # build the excitation gates
    for excitation_gate in total:
        partial = create_exponential_map(excitation_gate, num_qubits)
        U += partial
    E = tq.ExpectationValue(U=U, H=hamiltonian)
    result = tq.simulate(E)
    return result

In [7]:
opt_result = op.minimize(optimize, x0=[np.pi/3, np.pi/3], method="Nelder-Mead")
opt_result.fun

-1.7655643975022528

Now let us now introduce shot noise, for that we will add shots to the optimization function.

In [8]:
def optimize_noisy(input):
    unitary = of.uccsd_singlet_generator(input, 4, 2)
    unitary_jw = of.jordan_wigner(unitary)
    singles, doubles = [], []
    for element in unitary_jw.get_operators():
        gates = list(element.terms.keys())[0]
        # single excitation gates always have three single qubit gates
        # doubles always have four, this only holds in this specific example
        if len(gates) == 3:
            singles.append(element)
        else:
            doubles.append(element)

    total = doubles + singles

    num_qubits = 4
    num_electrons = 2
    U = tq.circuit.QCircuit()

    # initialize the electrons
    for electron in range(num_electrons):
        U += tq.gates.X(electron)

    # build the excitation gates
    for excitation_gate in total:
        partial = create_exponential_map(excitation_gate, num_qubits)
        U += partial
    E = tq.ExpectationValue(U=U, H=hamiltonian)
    
    result = tq.simulate(E, samples=1000)
    return result

In [9]:
opt_result_noisy = op.minimize(optimize_noisy, x0=[np.pi/4, np.pi/4], method="Nelder-Mead")
opt_result_noisy.fun

-1.7940000000000005

When we introduce sampling noise, we see that the optimization takes longer and that the result is less accurate. However by increasing the amount of samples we can increase the accuracy.